In [20]:
import pandas as pd
import numpy as np
import torch.nn as nn 
from sklearn.datasets import make_regression
from torch.utils.data import Dataset, DataLoader
import torch
from typing import Tuple
from sklearn.linear_model import LinearRegression, HuberRegressor
from sklearn.preprocessing import StandardScaler
import plotly.express as px
from pathlib import Path
from sklearn.metrics import root_mean_squared_error

In [21]:
fold = 4
data_folder = Path(
    "/home/thomas/repos/simplify_deployment/data/data_science/50_gen_s1/folds_converted_with_genome",
)
X_train = pd.read_parquet(data_folder/f"X_train_fold_{fold}.parquet")
y_train = pd.read_parquet(data_folder/f"y_train_fold_{fold}.parquet").squeeze()
X_test = pd.read_parquet(data_folder/f"X_test_fold_{fold}.parquet")
y_test = pd.read_parquet(data_folder/f"y_test_fold_{fold}.parquet").squeeze()
n_features = X_train.shape[1]

In [22]:
n_features

93

## Standard linear regression
Let's start by fitting a standard linear regression and plotting the results.

In [23]:
standard_model = LinearRegression()
standard_model.fit(X_train, y_train)
# huber_model = HuberRegressor(max_iter=1000)
# huber_model.fit(X_train, y_train)
prediction_df = pd.DataFrame(
    {
        "y_true": y_test,
        "y_pred_standard_model": standard_model.predict(X_test),
        # "y_pred_huber": huber_model.predict(X_test)
    },
    index = y_test.index
)
prediction_df

,y_true,y_pred_standard_model
2023-03-23 22:44:00+00:00,71.948,-0.506093
2023-03-23 22:59:00+00:00,85.727,82.152897
2023-03-23 23:14:00+00:00,-10.197,-28.967203
2023-03-23 23:29:00+00:00,45.033,38.841847
2023-03-23 23:44:00+00:00,93.805,88.554369
...,...,...
2023-04-18 21:29:00+00:00,-93.885,-36.492701
2023-04-18 21:44:00+00:00,17.815,-0.004208
2023-04-18 21:59:00+00:00,7.509,29.680265
2023-04-18 22:14:00+00:00,-184.809,-101.897864


In [24]:
prediction_df_molten = prediction_df.melt(ignore_index=False)
fig = px.line(
    prediction_df_molten,
    x = prediction_df_molten.index,
    y = "value",
    color = "variable",
)
fig.show()

## Custom loss model
This time we will fit a linear model with our custom loss function. It should perform less well in regards to RMSE as this it not what it optimizes for.
We should see less max errors, or less time above threshold, ... depending on what we prioritize in the loss function.

For the optimizer to work well, it is best to standardscale the data. This brings some added complexity in the code, but is necessary to speed up convergence.

Let's starts by setting up some classes we will need:

In [25]:
class Model(nn.Module):
    def __init__(
            self,
            n_features: int,
    ) -> None:
        super().__init__()
        self.fc1 = nn.Linear(
            in_features=n_features,
            out_features=1,
        )  # Just 1 fully connected layer without activation, i.e. a linear regression.

    def forward(
        self,  
        X: torch.Tensor,  
    ) -> torch.Tensor:
        y = self.fc1(X)
        return y.flatten()

In [26]:
class CustomLoss(nn.Module):
    def __init__(
            self,
            threshold: float = 0.685,
            weight_max_error: float = 1,
            weight_percentage_above_threshold:float = 1,
            weight_wrong_sign: float = 1,
            sigmoid_steepness: float = 1,
    ) -> None:
        super().__init__()
        self.steepness = sigmoid_steepness
        self.threshold = threshold
        # Normalize weights and assign them
        sum_weights = (
            weight_max_error
            + weight_percentage_above_threshold
            + weight_wrong_sign
        )
        self.weight_max_error = (
            weight_max_error / sum_weights
        )
        self.weight_percentage_above_threshold = (
            weight_percentage_above_threshold / sum_weights
        )
        self.weight_wrong_sign = (
            weight_wrong_sign / sum_weights
        )

    def forward(
            self, 
            inputs: torch.Tensor, 
            targets: torch.Tensor,
        ) -> torch.Tensor:

        residuals = targets - inputs
        # Maximum abs error
        max_error = residuals.abs().max()

        # Percentage of time above threshold value
        percentage_of_time_above_x = (
            1/(1+torch.e**(-self.steepness*(residuals.abs()-self.threshold)))
        ).mean()

        # Percentage of time wrong sign
        loss_percentage_of_time_wrong_sign = (
            1/(1+torch.e**(-self.steepness*(inputs*targets)))
        ).mean()
        
        # Total loss
        total_loss = (
            self.weight_max_error * max_error
            + self.weight_percentage_above_threshold * percentage_of_time_above_x
            + self.weight_wrong_sign * loss_percentage_of_time_wrong_sign
        )
        return total_loss

In [ ]:
class CustomLossRelu(nn.Module):
    def __init__(
            self,
            threshold: float = 0.685,
            weight_max_error: float = 1,
            weight_percentage_above_threshold:float = 1,
            weight_wrong_sign: float = 1,
            sigmoid_steepness: float = 1,
    ) -> None:
        super().__init__()
        self.steepness = sigmoid_steepness
        self.threshold = threshold
        # Normalize weights and assign them
        sum_weights = (
            weight_max_error
            + weight_percentage_above_threshold
            + weight_wrong_sign
        )
        self.weight_max_error = (
            weight_max_error / sum_weights
        )
        self.weight_percentage_above_threshold = (
            weight_percentage_above_threshold / sum_weights
        )
        self.weight_wrong_sign = (
            weight_wrong_sign / sum_weights
        )

    def forward(
            self, 
            inputs: torch.Tensor, 
            targets: torch.Tensor,
        ) -> torch.Tensor:

        residuals = targets - inputs
        # Maximum abs error
        max_error = residuals.abs().max()

        # Percentage of time above threshold value
        percentage_of_time_above_x = (
            1/(1+torch.e**(-self.steepness*(residuals.abs()-self.threshold)))
        ).mean()

        # Percentage of time wrong sign
        loss_percentage_of_time_wrong_sign = (
            1/(1+torch.e**(-self.steepness*(inputs*targets)))
        ).mean()
        
        # Total loss
        total_loss = (
            self.weight_max_error * max_error
            + self.weight_percentage_above_threshold * percentage_of_time_above_x
            + self.weight_wrong_sign * loss_percentage_of_time_wrong_sign
        )
        return total_loss

In [27]:
class CustomMSELoss(nn.Module):
    def __init__(
            self,
    ) -> None:
        super().__init__()


    def forward(
            self, 
            inputs: torch.Tensor, 
            targets: torch.Tensor,
        ) -> torch.Tensor:

        residuals = targets - inputs
        mse = (residuals**2).mean()
        return mse

In [28]:
class CustomDataset(Dataset):
    def __init__(
            self,
            X: torch.Tensor,
            y: torch.Tensor,
        ) -> None:
        self.X = X
        self.y = y

    def __len__(
            self
    ) -> int:
        return self.X.shape[0]
    
    def __getitem__(
            self,
            idx: int,
    ) -> Tuple[torch.Tensor, torch.Tensor]:
        X_item = self.X[idx,:]
        y_item = self.y[idx]
        return X_item, y_item

Now that we created our custom loss and model, we need to start prepping the data.

- Step 1: Standardscale X and y
- Step 2: Convert data to Tensors
- Step 3: Optimization loop on Tensors
- Step 4: Convert Tensors back and undo scaling so we can compare with previous results

As a lot of steps here might introduce bugs, a sanity check will be no luxury. We will start with a custom loop that also implements MSE loss. 
This should give the same result as the Sklearn implementation. If this is the case, we passed our sanity check

In [29]:
X_scaler = StandardScaler().fit(X_train)
y_scaler = StandardScaler().fit(y_train.to_frame())

X_train_scaled = torch.Tensor(
    X_scaler.transform(X_train),
).float()

X_test_scaled = torch.Tensor(
    X_scaler.transform(X_test),
).float()

y_train_scaled = torch.Tensor(
    y_scaler.transform(y_train.to_frame()).squeeze(), # Make target tensor unidimensional
).float()

y_test_scaled = torch.Tensor(
    y_scaler.transform(y_test.to_frame()).squeeze(),
).float()

Standard MSE loss, sanity check

In [30]:
epochs = 1
lr = 1e-4
batch_size = 10

dataloader = DataLoader(
    CustomDataset(X_train_scaled,y_train_scaled),
    batch_size=batch_size,
    shuffle=True,
)
model = Model(
    n_features=n_features,
)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = lr,
)
criterion = nn.HuberLoss()
for epoch in range(epochs):
    epoch_loss = 0
    for i,(X_batch, y_batch) in enumerate(dataloader):
        prediction = model(X_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        average_loss = epoch_loss/(i+1)
    print(f"Average epoch loss: {average_loss}")
    print(f"Epoch {epoch} done.")

Average epoch loss: 0.37263326181423373
Epoch 0 done.


In [31]:
model.fc1.weight.grad

tensor([[-0.1860, -0.1833, -0.1854, -0.1775,  0.0188,  0.1376, -0.1409, -0.1055,
          0.1531, -0.0902, -0.0803, -0.0647, -0.0731, -0.0958, -0.1059, -0.0438,
         -0.0674, -0.0663, -0.0719, -0.0584, -0.0757, -0.0644,  0.2207,  0.2618,
          0.2631,  0.2730,  0.2385,  0.1843,  0.2347,  0.1904,  0.1517, -0.2047,
         -0.1878, -0.2443,  0.0808,  0.0087,  0.0273,  0.1290,  0.0709,  0.2248,
          0.1745,  0.0406, -0.0763,  0.1036,  0.0454,  0.0886, -0.0971,  0.0245,
          0.0179, -0.1837,  0.0968,  0.0816, -0.0267, -0.2067, -0.1611, -0.0493,
          0.2545,  0.0764, -0.0334,  0.0150,  0.1289,  0.1213, -0.1199,  0.0519,
         -0.1309, -0.3235, -0.1366,  0.1792,  0.0083, -0.0431, -0.1897,  0.0995,
          0.3308,  0.1486,  0.0697, -0.0049, -0.0130, -0.0731, -0.0573, -0.2462,
         -0.2662,  0.1261, -0.0596, -0.1123,  0.0878,  0.0985, -0.1110,  0.0646,
         -0.0626,  0.2001,  0.1178, -0.0884, -0.0877]])

In [32]:
prediction_df["y_pred_sanity_check"] = y_scaler.inverse_transform(
    model(X_test_scaled).detach().numpy().reshape(-1,1),
)
prediction_df


,y_true,y_pred_standard_model,y_pred_sanity_check
2023-03-23 22:44:00+00:00,71.948,-0.506093,-8.978985
2023-03-23 22:59:00+00:00,85.727,82.152897,19.751732
2023-03-23 23:14:00+00:00,-10.197,-28.967203,-50.947582
2023-03-23 23:29:00+00:00,45.033,38.841847,45.079945
2023-03-23 23:44:00+00:00,93.805,88.554369,22.979401
...,...,...,...
2023-04-18 21:29:00+00:00,-93.885,-36.492701,-54.128403
2023-04-18 21:44:00+00:00,17.815,-0.004208,-78.917961
2023-04-18 21:59:00+00:00,7.509,29.680265,-26.289423
2023-04-18 22:14:00+00:00,-184.809,-101.897864,64.802551


In [33]:
prediction_df_molten = prediction_df.melt(ignore_index=False)
fig = px.line(
    prediction_df_molten,
    x = prediction_df_molten.index,
    y = "value",
    color = "variable",
)
fig.show()

We can see that the sanity check is still not totally equal to the sklearn model. This might be due to the way the model is trained. 
In sklearn we optimize the RMSE over all in one go. With our torch loop we work per batch and minimize the rmse per batch. We see that in practice the predictions are however not too far off. In theory model with an optimal rmse per batch will also have a global optimal RMSE. If we really really want to check this, we can give it a lot more epochs.

Out of interest the RMSE values for both on the test data:

In [34]:
rmse_standard_model = root_mean_squared_error(
    prediction_df["y_true"],
    prediction_df["y_pred_standard_model"],
)
rmse_torch_model = root_mean_squared_error(
    prediction_df["y_true"],
    prediction_df["y_pred_sanity_check"],
)
print(f"RMSE of sklearn model is {rmse_standard_model}")
print(f"RMSE of torch model is {rmse_torch_model}")

RMSE of sklearn model is 135.49966253749366
RMSE of torch model is 162.94611798503058


## Custom loss model
Now what you have been actually waiting for, the model with the custom loss function. 


In [35]:
epochs = 500
lr = 1e-4
batch_size = 10

dataloader = DataLoader(
    CustomDataset(X_train_scaled,y_train_scaled),
    batch_size=batch_size,
    shuffle=True,
)
model_custom_loss = Model(
    n_features=n_features,
)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr = lr,
)
# Part of the loss function will be how much time the absolute value of the residuals is above a threshold, 117 MW if i'm not mistaken.
# However we converted our y values in z-scores as this helps the optimizer a lot. So we will too for this threshold.
# A mistake to avoid is to just convert this threshold in the z score of the threshold. 
# E.g. imagine: sigma = 10 (standarddeviation), mu=5 (mean). 
# Problem: we have 2 values, say 20 & 15 and we want to check if their residuals are above or equal to a threshold of 5
# In non z-score land this is easy: 20-15 = 5, so equal to the threshold.
# In z-score land we got (20-5)/10 - (15-5)/10 = 0.5 -> This is not the z-score of 5! (which is zero) 
# So make sure to not convert the threshold to its z-score.
# But then how do we convert it? With a small bit of calculus we can show that the threshold in z-score land is the normal threshold/sigma
# E.g. 5/10 would be 0.5 in z-score land. Which is indeed correct for our example.
threshold = 117
converted_threshold = threshold / np.std(y_train)
criterion = CustomLoss(
    threshold=converted_threshold,
    weight_wrong_sign=0,
    weight_max_error=1,
    weight_percentage_above_threshold=0,
) # This is our custom loss. You can give random weights to the different factors of the loss function. 
# Or feel free to implement something yourself in the loss.
for epoch in range(epochs):
    epoch_loss = 0
    for i,(X_batch, y_batch) in enumerate(dataloader):
        prediction = model_custom_loss(X_batch)
        optimizer.zero_grad()
        loss = criterion(prediction, y_batch)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
        average_loss = epoch_loss/(i+1)
    print(f"Average epoch loss: {average_loss}")
    print(f"Epoch {epoch} done.")

Average epoch loss: 2.1143757520582187
Epoch 0 done.
Average epoch loss: 2.1231128753473363
Epoch 1 done.
Average epoch loss: 2.1126181495121936
Epoch 2 done.
Average epoch loss: 2.1109091138862297
Epoch 3 done.
Average epoch loss: 2.11370132959476
Epoch 4 done.
Average epoch loss: 2.1084076136802183
Epoch 5 done.
Average epoch loss: 2.1238806624755715
Epoch 6 done.
Average epoch loss: 2.1117460160431536
Epoch 7 done.
Average epoch loss: 2.1174985262367763
Epoch 8 done.
Average epoch loss: 2.1309524599011196
Epoch 9 done.
Average epoch loss: 2.1324510094568585
Epoch 10 done.


KeyboardInterrupt: 

In [17]:
prediction_df["y_pred_custom_loss"] = y_scaler.inverse_transform(
    model_custom_loss(X_test_scaled).detach().numpy().reshape(-1,1),
)
prediction_df

,y_true,y_pred_standard_model,y_pred_huber,y_pred_sanity_check,y_pred_custom_loss
2023-03-23 22:44:00+00:00,71.948,-0.506093,15.108088,23.341362,-8.818180
2023-03-23 22:59:00+00:00,85.727,82.152897,74.823266,-5.654055,-141.436584
2023-03-23 23:14:00+00:00,-10.197,-28.967203,-28.300149,68.736046,-9.992126
2023-03-23 23:29:00+00:00,45.033,38.841847,44.622261,100.453209,-9.849909
2023-03-23 23:44:00+00:00,93.805,88.554369,88.623767,101.859024,-50.748516
...,...,...,...,...,...
2023-04-18 21:29:00+00:00,-93.885,-36.492701,-30.384722,26.841042,-42.295963
2023-04-18 21:44:00+00:00,17.815,-0.004208,5.840203,151.671082,27.450474
2023-04-18 21:59:00+00:00,7.509,29.680265,33.455033,64.415695,2.763256
2023-04-18 22:14:00+00:00,-184.809,-101.897864,-109.869062,19.282597,22.560619


In [18]:
custom_rmse = root_mean_squared_error(
    prediction_df["y_true"],
    prediction_df["y_pred_custom_loss"],
)
print(f"Rmse of custom model is {custom_rmse}")

Rmse of custom model is 228.8356265860311


In [19]:
prediction_df_molten = prediction_df.melt(ignore_index=False)
fig = px.line(
    prediction_df_molten,
    x = prediction_df_molten.index,
    y = "value",
    color = "variable",
)
fig.show()